# Data Science Project - Detecting Fraudulent Credit Card Transactions

The research question here is to investigate whether we can determine a credit card transation to be fraudulent, using the Credit Card Fraud Detection dataset from Kaggle.

First we need to import necessary libraries and load in the data. Then do some early exploratory data analysis to better understand the data.

In [ ]:
""" Identify whether a credit card transaction is fraudulent or not. Using credit card transaction data from Kaggle """

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import imblearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import sklearn.metrics

# Load the test and training data
train_raw_df = pd.read_csv(".\Data\creditcard_train.csv")
test_raw_df = pd.read_csv(".\Data\creditcard_test.csv")

# See how many rows and columns there are
train_raw_df.shape
test_raw_df.shape

# Look for null values and make sure data types are matching
print(train_raw_df.info())
print(test_raw_df.info())

# Get a brief visual look at the actual values in the data and make some initial deductions
train_raw_df.head()




First inspection seems to show that we are dealing with numerical data in our 30 features, and a categorical label in our 'Class' column with just two classes "1" and "0". 

There are fortunately no null or missing values in either training or test set.

We can also see that features 'V1 - V28' might already been feature scaled in some way, where as 'Time' and 'Amount' have not. Let's use .describe() on every column to check this.

In [ ]:
for column in train_raw_df:
    print(train_raw_df[column].describe(), "\n")

The prints from above confirm these initial thoughts, because the mean for all of the columns from 'V1' to 'V28' are extremely close to zero, suggesting that the data has been standardized (z-score normalised). 

It therefore makes sense to use this same type of normalisation on the non-feature scaled features, 'Time' and 'Amount' but only when we are using ML models to classify the data. 

In the meantime, let us continue with further exploratory data analysis.

In [ ]:
# Look for duplicate values
print("Train duplicates:", train_raw_df.duplicated().sum())
print("Test duplicates:", test_raw_df.duplicated().sum())

train_duplicates = train_raw_df[train_raw_df.duplicated()]
train_duplicates.sort_values("Time")

It appears we have a number of duplicates in our datasets. Unfortunately, our data does not contain a clearly identifiable primary key such as 'Transaction ID'. If that was the case then we could simply remove duplicates which shared the same transaction ID.

Looking at the documentation for the dataset (Source: https://www.kaggle.com/mlg-ulb/creditcardfraud) we can see that V1-V28 were likely to have been anonymised for the sake of protecting user's identity. This means that it is likely that the values in these fields combined could be enough to uniquely identity a person. Therefore, it makes it highly improbable for all the values in V1-28 AS WELL AS the values in time and amount to all be exactly the same in more than one entry. On this basis, it seems sensible to remove the duplicate values.

In [ ]:
# Remove duplicate data
train_df = train_raw_df.drop_duplicates()
test_df = test_raw_df.drop_duplicates()
print(train_raw_df.shape)
print(train_df.shape)


In [ ]:
# Convert the 'Class' colum from int64 to category as we know it is a categorical variable
test_df["Class"] = test_df['Class'].astype('category')
train_df["Class"] = train_df['Class'].astype('category')
train_df["Class"]



Now we should try to identify influencial variables by performing futher exploratory analysis whilst also cleaning up the data.

Earlier we saw a very high value for the 'max' in the amount column so let's take a lot at the distribution of data in this column.


In [ ]:
# Boxplot for Amount column
fig = px.box(train_df, y="Amount")
fig.show()


In [ ]:
# Take a look at the two largest values for Amount
outliers = train_df.nlargest(2, "Amount")
# Since these outliers are part of the majority class, getting rid of them should not have a negative effect
train_df = train_df[~train_df.isin(outliers)].dropna(how="all")
fig = px.box(train_df, y="Amount", title="Boxplot for Amount")
fig.show()


In [ ]:
# View the distribution of the other features
fig = px.box(train_df, x=["V" + str(i) for i in range(1,29)], title="Boxplot of all the anonymous features")
#fig.show() don't show due to computer lag


## Removal of outliers

Since a few of the anonymous features appear to have a couple outliers, we can go through all of them and drop the two largest and two smallest, but we should only do this if they are memebr of the majority class (Class=0) since this is something we will be doing anyway when we undersample.

In [ ]:
# Extract the majority class from the train_df
class_0_train_df = train_df[train_df["Class"] == 0]

# Go through the columns of the df
for series in class_0_train_df:
    if series != "Time" and series != "Amount" and series != "Class":
        # Find the upper and lower 0.1%
        q_low = class_0_train_df[series].quantile(0.001)
        q_hi  = class_0_train_df[series].quantile(0.999)
        # Remove these extreme values
        no_outlier_train_df = class_0_train_df[(class_0_train_df[series] < q_hi) & (class_0_train_df[series] > q_low)]

# Add all the minority class examples back in
no_outlier_train_df = no_outlier_train_df.append(train_df[train_df["Class"] == 1])

print(no_outlier_train_df.shape)
print(train_df.shape)
   

Now we want perform dimensionality reduction since we have a large number of dimensions. PCA will help to reduce the dimensions and enable us to visualise the data better.

In [ ]:
# Standardise "Time" and "Amount" since the other features are already standardised
norm_train_df = no_outlier_train_df
def standardise(X):
    X_std = X - np.mean(X, axis=0)      # Subtract the mean of the feature from each example
    X_std = np.divide(X_std, np.std(X_std, axis=0))     # Divide each example by the range of the feature
    return X_std

norm_train_df["Time"] = standardise(no_outlier_train_df["Time"])
norm_train_df["Amount"] = standardise(no_outlier_train_df["Amount"])
# Separate X and Y from dataset
train_X_df = norm_train_df.iloc[:, :-1]
train_y_df = norm_train_df["Class"]
train_X, train_y = np.array(train_X_df), np.array(train_y_df)
# Compute covariance matrix
cov_mat = np.matmul(train_X.T, train_X)
# Compute eigenvectors and eigenvalues
eig_val, eig_vec = np.linalg.eig(cov_mat)
print("Eigenvectors:", eig_vec, "\nEigenvalues:", eig_val)

Now we can find out how much each eigenvector contributes to the overall variance by comparing the size of their eigenvalues. This can also be shown in a plot.

In [ ]:
# Majority of code taken from https://github.com/AI-Core/Practical-ML-DS/blob/master/Chapter%202.%20Python%20for%20Data%20Scientists/Module%205.%20Unsupervised%20Feature%20Representations/0.%20PCA%20and%20t-SNE.ipynb

var_percent = [(i/sum(eig_val)) * 100 for i in eig_val]   # calculate the percentage variance of the data which this eigenvalue accounts for
cum_var_percent = np.cumsum(var_percent)    # make a vector of the cumulative sum of the variance percentages

fig = plt.figure(figsize=(22,10))      
ax =  fig.add_subplot(111)      # add an axis
plt.title('Variance along principal components')
ax.grid()
plt.xlabel('Principal Component')
plt.ylabel('Percentage total variance accounted for')

ax.plot(cum_var_percent, '-ro')     # plot the cumulative sum of the variances accounted for by each eigenvector
ax.bar(range(len(eig_val)), var_percent) # position, height # show how much variance individual eig accounts for
plt.xticks(np.arange(len(eig_val)), ('PC{}'.format(i) for i in range(len(eig_val))))  # set the xticks to 'PC1' etcx
plt.show()

In [ ]:
# Reduce data to 3 dimensions
W = eig_vec[:, :3]
reduced_train_X = np.matmul(train_X, W)
reduced_train_X


Now we can plot the data

In [ ]:
# Assign different colours to each class
colour_dict = {0:'r', 1:'b'}     
colour_list = [colour_dict[i] for i in list(train_y)] 

fig = plt.figure(figsize=(25,15))
plt.grid()
ax = fig.add_subplot(111, projection='3d')      # add a 3d set of axes
ax.scatter(reduced_train_X[:, 0], reduced_train_X[:, 1], reduced_train_X[:, 2], c=colour_list)    # scatter plot our 3d data
plt.xlabel('PC1 value')
plt.ylabel('PC2 value')
ax.set_zlabel('PC3 value')
plt.show()

## Imbalanced Data - Undersampling vs Oversampling

Imbalanced data is when the distribution of classes is uneven, and there is a clear majority and minority class. Undersampling involves removing examples in the majority class to help balance the data, whereas oversampling involves duplicating example from the minority class. The examples to be removed or duplicated are selected at random, which is why these methods are types of random resampling. These naive methods also assume nothing about the data, which makes them fast and easy to use as no heuristics are used.  (Source: https://machinelearningmastery.com/random-oversampling-and-undersampling-for-imbalanced-classification/)

Random oversampling increases the chance of overfitting to the minority class, as the number of their cases have been artificially inflated. In contrast, random undersampling increases the chance to reduce the performance of your model as you might lose key information-rich examples from the dataset.

## Identify and discuss at least 2 suitable evaluation metrics for this task. Then classify the data.

Since we are working with imbalanced data, it does not make sense to use accuracy as an evaluation metric. This is because if the system just predicted everything to be negative (i.e. class = 0) it would still get a high accuracy score. Instead we should look at precision, recall, and the F1 score which is a combination of the previous two. 

Since we have a low number of overall positive cases (i.e. where the class = 1), recall and F1 score will be the two most important metrics here and therefore what will be taken into account. This is because recall is the number of true positives divided by the total number of actual positives in the dataset. This means it will give a low score if the model gives a lot of false negatives. F1 score is good because it combines both precision and recall to give a good overall score. (Source: https://machinelearningmastery.com/classification-accuracy-is-not-enough-more-performance-measures-you-can-use/)




In [ ]:
# standardise the test dataset too
norm_test_df = test_df
norm_test_df["Time"] = standardise(test_df["Time"])
norm_test_df["Amount"] = standardise(test_df["Amount"])

# separate into X and labels
test_X_df = norm_test_df.iloc[:,:-1]
test_y_df = norm_test_df["Class"]

# Find out the current ratio of minority to majority class examples
print(np.sum(train_y == 0)/np.sum(train_y == 1)) 
# Use the Imblearn library to resample the dataset
over = imblearn.over_sampling.RandomOverSampler(sampling_strategy=0.02) # Oversample until the ratio is 1:50 or 0.02
resampled_X, resampled_y = over.fit_resample(train_X, train_y)

under = imblearn.under_sampling.RandomUnderSampler(sampling_strategy=0.5) # Undersample until the ratio is 1:2 or 0.5
resampled_X, resampled_y = under.fit_resample(resampled_X, resampled_y)

# print new ratio of minority to majority class (should be 2 majority class cases for every 1 minority class case)
print(np.sum(resampled_y == 0)/np.sum(resampled_y == 1)) 



Now that we have resampled our dataset to make it more balanced, and have also normalised the data earlier, we are ready to run some models to classify the data. The model's perfomance will be based on recall and F1 score.

In [ ]:
# Create a validation dataset from the version of the training data BEFORE resampling
dont_use_X, valid_X, dont_use_y, valid_y = train_test_split(train_X, train_y)

# Logistic regression
model1, model2, model3 = LogisticRegression(solver="sag"), LogisticRegression(solver="saga"), LogisticRegression(solver="liblinear")
for model in [model1, model2, model3]:
    model.fit(train_X, train_y)
    pred_y = model.predict(valid_X)
    print("Recall score:", sklearn.metrics.recall_score(valid_y, pred_y))
    print("F1 score:", sklearn.metrics.f1_score(valid_y, pred_y))


In [ ]:
# Random forests
for max_depth in range(1,10):
    model = RandomForestClassifier(max_depth=max_depth)
    model.fit(train_X, train_y)
    pred_y = model.predict(valid_X)
    print("With trees of max depth =", max_depth, "\nRecall score:", sklearn.metrics.recall_score(valid_y, pred_y))
    print("F1 score:", sklearn.metrics.f1_score(valid_y, pred_y))

In [ ]:
# K nearest neighbours
for n_neighbours in range(1,20):
    model = KNeighborsClassifier(n_neighbors=n_neighbours)
    model.fit(train_X, train_y)
    pred_y = model.predict(valid_X)
    print("With number of nearest neighbours =", n_neighbours, "\nRecall score:", sklearn.metrics.recall_score(valid_y, pred_y))
    print("F1 score:", sklearn.metrics.f1_score(valid_y, pred_y))


Of the models tried, the best performing model was random forests with a max depth of 9. Let's now try this on the test data.

In [ ]:
model = RandomForestClassifier(max_depth=9)
model.fit(train_X, train_y)
pred_y = model.predict(test_X)
print("With trees of max depth =", max_depth, "\nRecall score:", sklearn.metrics.recall_score(test_y, pred_y))
print("F1 score:", sklearn.metrics.f1_score(test_y, pred_y))

# Show these results in a confusion matrix
sklearn.metrics.plot_confusion_matrix(model, test_X, test_y)
plt.show()


## Identify the top 8 most influential variables in the dataset.

This can be done by looking at the PCA graph already shown earlier and looking at the 8 largest bars representing how much variance each variable accounts for


In [ ]:
# Redo the plot of the PCAs from earlier

var_percent = [(i/sum(eig_val)) * 100 for i in eig_val]   # calculate the percentage variance of the data which this eigenvalue accounts for
cum_var_percent = np.cumsum(var_percent)    # make a vector of the cumulative sum of the variance percentages

fig = plt.figure(figsize=(22,10))      
ax =  fig.add_subplot(111)      # add an axis
plt.title('Variance along principal components')
ax.grid()
plt.xlabel('Principal Component')
plt.ylabel('Percentage total variance accounted for')

ax.plot(cum_var_percent, '-ro')     # plot the cumulative sum of the variances accounted for by each eigenvector
ax.bar(range(len(eig_val)), var_percent) # position, height # show how much variance individual eig accounts for
plt.xticks(np.arange(len(eig_val)), ('PC{}'.format(i) for i in range(len(eig_val))))  # set the xticks to 'PC1' etcx
plt.show()

Here we can see that the first 8 principal components account for most of the variance. Therefore this means that the 8 most influencial variables are the first 8: Time, V1, V2, V3, V4, V5, V6, V7 (in descending order of importance).